In [80]:
import re

In [81]:
# FUNKCJE POMOCNICZE

table_data = """
UUU F      CUU L      AUU I      GUU V
UUC F      CUC L      AUC I      GUC V
UUA L      CUA L      AUA I      GUA V
UUG L      CUG L      AUG M      GUG V
UCU S      CCU P      ACU T      GCU A
UCC S      CCC P      ACC T      GCC A
UCA S      CCA P      ACA T      GCA A
UCG S      CCG P      ACG T      GCG A
UAU Y      CAU H      AAU N      GAU D
UAC Y      CAC H      AAC N      GAC D
UAA *      CAA Q      AAA K      GAA E
UAG *      CAG Q      AAG K      GAG E
UGU C      CGU R      AGU S      GGU G
UGC C      CGC R      AGC S      GGC G
UGA *      CGA R      AGA R      GGA G
UGG W      CGG R      AGG R      GGG G
""".split()

codon_2_aa = dict(zip(table_data[::2], table_data[1::2]))

def translate2(seq):
    return "".join([codon_2_aa.get(seq[i:i+3], '') for i in range(0, len(seq), 3)])

def transcribe2(seq):
    return seq.replace("T", "U")

def rev_comp2(seq):
    return seq.translate(str.maketrans("ATCG", "TAGC"))[::-1]

In [82]:
dna_seq = "AGCCATGTAGCTAACTCAGGTTACATGGGGATGACCCCGCGACTTGGATTAGAGTCTCTTTTGGAATAAGCCTGAATGATCCGAGTAGCATCTCAG"
orf_pattern = r'(?=(AUG(?:...)*?)(?=UAG|UGA|UAA))' # można re.compile, ale daje to bardzo mało..
orfs = []

for frame_shift in range(3):
    orfs.append(re.findall(orf_pattern, transcribe2(dna_seq)[frame_shift:]))
    orfs.append(re.findall(orf_pattern, transcribe2(rev_comp2(dna_seq))[frame_shift:]))

print(set([translate2(seq) for seq in sum(orfs, [])]))

{'MLLGSFRLIPKETLIQVAGSSPCNLS', 'M', 'MGMTPRLGLESLLE', 'MTPRLGLESLLE'}


**?= positive lookahead assertion** - dopasowane znaki nie zostaną zużyte, pozwoli to na wykrycie również nakładających się orf'ów.

### Rozwiązanie przy użyciu BioPython

In [83]:
from Bio.Seq import Seq

seq = Seq("AGCCATGTAGCTAACTCAGGTTACATGGGGATGACCCCGCGACTTGGATTAGAGTCTCTTTTGGAATAAGCCTGAATGATCCGAGTAGCATCTCAG")
regex = "(?=(M.*?\*))" # \* bo * ma być traktowana jako zwykły symbol (\ escape character)
orfs = []

for frame_shift in range(3):
    orfs.append(re.findall(regex, str(seq.transcribe()[frame_shift:].translate()))) # można bezpośrednio translate..
    orfs.append(re.findall(regex, str(seq.reverse_complement().transcribe()[frame_shift:].translate())))

print(set(sum(orfs, []))) # sum(lista, []) połączy elementy list listy w liste elementów list listy ;)

{'MLLGSFRLIPKETLIQVAGSSPCNLS*', 'MTPRLGLESLLE*', 'MGMTPRLGLESLLE*', 'M*'}


Oczekiwane wyniki:

 * MLLGSFRLIPKETLIQVAGSSPCNLS
 * M
 * MGMTPRLGLESLLE
 * MTPRLGLESLLE